# Processing scraped jobs

Once job details and descriptions are scraped a few more things need to happen:

* compare today's vs. yesterday's jobs
* when new job comes in, add day tracker (days = 0)
* if job not new, days += 1

**Import some libraries**

In [105]:
import pandas as pd
import numpy as np

# plotting pacakges
import matplotlib.pyplot as plt
import seaborn as sns
import plotly as py
%matplotlib inline

# ignore matplotlib/seaborn deprecation warning... it just looks ugly
import warnings
warnings.filterwarnings("ignore", message="The 'normed' kwarg is deprecated, and has been ")

# assortment of other libraries
import re
import time
from nltk.corpus import stopwords
import string
from collections import Counter
import pandas as pd
import numpy as np
import arrow

In [135]:
df_test = pd.read_csv("./data/glassdoor-df-06-05-2018.csv", encoding="ISO-8859-1")
df_test2 = pd.read_csv("./data/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")
#df_new = pd.read_csv("./data/glassdoor-df-06-06-2018.csv", encoding="ISO-8859-1")

In [121]:
print(df_test.shape)
df_test.head(n=5)

(835, 5)


,position,company,location,description,link
0,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...
1,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...
2,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...
3,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...
4,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...


In [122]:
print(df_test2.shape)
df_test2.head(n=5)

(5809, 5)


,position,company,location,description,link
0,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...
1,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...
2,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...
3,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...
4,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...


In [153]:
df_test.iloc[[1]]['link'].values[0]

'https://www.glassdoor.com/partner/jobListing.htm?pos=102&ao=133029&s=58&guid=00000163d279fd7d920c9d4362241cf2&src=GD_JOB_VIEW&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_74ca3bc8&cb=1528244619445&jobListingId=2697694396'

In [181]:
df_test.iloc[[2]]['link'].values[0]

'https://www.glassdoor.com/partner/jobListing.htm?pos=103&ao=142359&s=58&guid=00000163d279fd7d920c9d4362241cf2&src=GD_JOB_VIEW&t=SR&extid=1&exst=OL&ist=&ast=OL&vt=w&slr=true&rtp=0&cs=1_b69b626c&cb=1528244622198&jobListingId=2748216148'

Links change even for same job. Depends where it is in search result list.

In [192]:
df_test.iloc[:60]

,position,company,location,description,link,days_posted,date_posted,applied,date_applied
0,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
1,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
2,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
3,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
4,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
5,"Quantitative Researcher, Growth - University Grad",Facebook,"Menlo Park, CA",Facebook's mission is to give people the power...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
6,New Grad - Research Scientist (PhD),Visa,"Palo Alto, CA","Company DescriptionCommon Purpose, Uncommon Op...",https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
7,Machine Learning Scientist (University Grad),Capital Games,"Redwood City, CA",Electronic Arts Inc. is a leading global inter...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
8,Machine Learning / AI Research Scientist (Grad),Analog Devices,Massachusetts,Analog Devices (NASDAQ: ADI) designs and manuf...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN
9,NYL Post Grad Internship: Data Scientist/Stati...,New York Life Insurance Co,"New York, NY",New York Life Insurance Company (âNew York L...,https://www.glassdoor.com/partner/jobListing.h...,0,06-06-2018,No,NaN


In [195]:
dups = df_test2.duplicated(subset=['position','company','location','description'])

In [206]:
Counter(dups)

Counter({False: 4906, True: 903})

In [207]:
df_test2 = df_test2[~dups]

In [202]:
without_dup.head(n=5)

,position,company,location,description,link
0,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...
1,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...
2,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...
3,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...
4,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...


**Add new columns to dataframe**

* days_since_posting
* date posted
* date ended
* applied
* date_applied

In [216]:
def initialize_first_df(df):
    
    # get rid of duplicates
    dups = df.duplicated(subset=['position','company','location','description'])
    df = df[~dups]
    
    # add some new columns of interest
    df['days_posted'] = 0
    df['date_posted'] = arrow.now().format('MM-DD-YYYY')
    df['applied'] = "No"
    df['date_applied'] = np.nan
    
    return df
    

In [214]:
df_test = initialize_first_df(df_test)

In [215]:
print(df_test.shape)
print(df_test2.shape)

(51, 9)
(4906, 5)


In [217]:
def combine_dfs(df_old, df_new):
    
    check = 0   
    
    #prepare df_new
    dups = df_new.duplicated(subset=['position','company','location','description'])
    df_new = df_new[~dups]
    
    # full outer join on old and new df (and get rid of duplicate columns)
    merged = df_new.merge(df_old, on=['company','position','location','description'], how="outer", indicator=True)
    merged = merged.drop(columns=['link_x'])
    merged = merged.rename(columns= {'link_y':'link'})
    
    # save old jobs to archive
    archive = merged[merged['_merge']=='right_only']
    merged = merged.drop(merged[merged['_merge']=='right_only'].index)
    
    # days_posted
    merged.loc[merged['_merge']=='both',('days_posted')] += 1
    merged.loc[merged['_merge']=='left_only',('days_posted')] = 0 
    
    # date_posted
    merged.loc[merged['_merge']=='left_only',('date_posted')] = arrow.now().format('MM-DD-YYYY') 
    
    # applied
    merged.loc[merged['_merge']=='left_only',('applied')] = "No"   

    return merged, archive

In [218]:
new_df, archive_df = combine_dfs(df_test, df_test2)

In [219]:
new_df.head(n=5)

,position,company,location,description,link,days_posted,date_posted,applied,date_applied,_merge
0,Data Scientist - New Grad,Yelp,"San Francisco, CA","At Yelp, we see our 148 million reviews not on...",https://www.glassdoor.com/partner/jobListing.h...,1.0,06-06-2018,No,NaN,both
1,Data Scientist - New Grad,"Quora, Inc.","Mountain View, CA","Because Quora is such a data-driven company, o...",https://www.glassdoor.com/partner/jobListing.h...,1.0,06-06-2018,No,NaN,both
2,Machine Learning Scientist (University Grad),Electronic Arts,"Redwood City, CA",We are EA\r\n\r\nAnd we make games how cool is...,https://www.glassdoor.com/partner/jobListing.h...,1.0,06-06-2018,No,NaN,both
3,New Grad - Data Scientist,Viasat,"Boston, MA",Job ResponsibilitiesBecome a personal contribu...,https://www.glassdoor.com/partner/jobListing.h...,1.0,06-06-2018,No,NaN,both
4,Data Scientist,Expedia,"Bellevue, WA",ExpediaWe are looking for a data scientist to ...,https://www.glassdoor.com/partner/jobListing.h...,1.0,06-06-2018,No,NaN,both


In [220]:
Counter(new_df['_merge'].tolist())

Counter({'both': 32, 'left_only': 4874})

**Filter jobs by location and job title**

In [227]:
def filter_by_location(df, states_of_interest):

    locations = df['location'].tolist()
    filter_index = []

    for location in locations:

        try:
            state = location.split(',')[1].strip()
            if state in states_of_interest:
                filter_index.append(True)
            else:
                filter_index.append(False)
        except:
            if location == 'Remote':
                filter_index.append(True)
            else:
                filter_index.append(False)

    return df[filter_index]

def filter_by_position_title(df, filter_words):
    
    filter_jobs = []

    jobs = df['position'].tolist()

    for job in jobs:
        job_words = job.split(" ")
        for word in job_words:
            if word.lower() in filter_words:
                include = False
                break;
            else:
                include = True
        if include:
            filter_jobs.append(True)
        else:
            filter_jobs.append(False)
    return df[filter_jobs]

In [ ]:
test.shape

In [228]:
states_of_interest = ['CA', 'CO', 'HI', 'IL', 'ME', 'MI', 'NV', 'NY', 'OH', 'OR', 'TX', 'WA', 'Remote']

test = filter_by_location(new_df, states_of_interest)

In [229]:
test.shape

(2804, 10)

In [230]:
filter_words = ['senior', 'manager', 'director', 'architect', 'sr.', 'sr', 'lead', 'phd', 'ph.d.', 'phd.']

test = filter_by_position_title(test, filter_words)

In [231]:
test.shape

(2239, 10)

**Filter by words in description**

Main ideas:

1) Rank jobs 1-3 (1 highest).


2) Filter on some ideas.

* if 5+ years experience: rank 3
* if Phd and not Master's: rank 3
* if no mention of education word but experience 2+: rank 2
* if master's and not phd: rank 1
* if bachelor and/or master: rank 1

In [ ]:
test_strings = ["Responsibilities include Phd or Master's", "experience require P.hd.", 
                "Must like a master", "do you have masters or P.hd?",
                "5+ years experience with a phd in masters", "2 year experience in something",
                "5 year experience with master's and phd."]

In [ ]:
re.search(,)

**Parse the job descriptions for key word counts**

In [ ]:
def parse_top_terms_glassdoor(job_descriptions):
    
    stop_words = set(stopwords.words('english'))

    job_tokens = []
    
    for job in job_descriptions:
        soup = BeautifulSoup(job, 'html.parser')
        words = soup.find('div', id='JobDescriptionContainer')
        words = words.text.strip().replace('\n', ' ').translate(string.punctuation)
        words = re.sub(r'[^\w\s]', '', words)
        filtered_words = list(filter(lambda w: not w in stop_words, words.lower().split()))
        job_tokens = job_tokens + filtered_words
        
    counted_words = Counter(job_tokens)
    
    return(counted_words)

In [94]:
df_test = compare_days(df_old, df_new)

('YapStone', 'Data Scientist')
('eBay Inc.', 'Sr. Data Scientist')
('Reali', 'Product Manager')
('eBay Inc.', 'Data Scientist, Marketing')
('Esri', 'Data Scientist')
('Axia Technologies', 'Senior Product Manager')
('Trianz', 'Python Full Stack Developer')
('realtor.com', 'Sr / Staff Software Engineer')
('Entytle', '\r\n\r\n                        Data Analyst\r\n')
('Match', 'Growth Marketer (Growth Hacker)')
('Ancestry', 'Quality Engineer - Data Science Engineering Team')
('Esri', 'Data Science Product Engineer - ArcGIS API for Python')
('eBay Inc.', 'Staff Data Scientist')
('M Theory Solutions', 'Data Scientist / Predictive Modeler')
('eBay Inc.', 'Data Scientist/Applied Researcher')
('Accenture', 'Data Scientist Associate Principal')
('Metromile', 'Director of Analytics')
('eBay Inc.', 'Senior Data Scientist - GBH')
('Metromile', 'Senior Product Manager, Post-Purchase')
('eBay Inc.', 'Data Scientist 2')
Counter({'No': 519, 'Yes': 386})


In [86]:
Counter(df_test[(df_test['time'].notnull()) & (df_test['New']=="Yes")]['time'].tolist())

Counter({0.0: 6,
         1.0: 3,
         4.0: 10,
         5.0: 1,
         6.0: 4,
         7.0: 9,
         8.0: 19,
         9.0: 12,
         10.0: 26,
         11.0: 4,
         12.0: 3,
         13.0: 6,
         14.0: 9,
         15.0: 20,
         16.0: 19,
         17.0: 22,
         18.0: 7,
         20.0: 1,
         21.0: 14,
         22.0: 18,
         23.0: 6,
         24.0: 12,
         25.0: 5,
         27.0: 4,
         28.0: 9})